In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

from pythainlp import word_tokenize
import re

import tensorflow as tf

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras import *
from keras.layers import *

from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('main_suicidal_data.csv')
df.head(5)

,Unnamed: 0,DateTime,Uid,Tweet,Label (Specialist),HashTag
0,0,18/5/2021 07:24,a38755f80aee,ถ้าฉันตายไปจริงๆก็คงดีสินะ,Level 2,#โรคซึมเศร้า
1,1,20/5/2021 16:06,a38755f80aee,เหนื่อยเหนื่อยจนจะตายอยู่แล้วมันไมกันนะนั่งเล่...,Level 1,#โรคซึมเศร้า
2,2,16/6/2021 07:00,9b7d113b270a,พยายามที่จะมีชีวิตอยู่แต่ก็เหนื่อยกับโลกนี้เหล...,Level 1,#โรคซึมเศร้า
3,3,28/5/2021 18:43,0b981040c16d,ชีวิตก็มีอยู่แค่นี้อยากหายไปแล้วนะ,Level 1,#โรคซึมเศร้า
4,4,13/7/2021 18:06,937c5794646d,วันวุ่นๆของวัยรุ่นกินยาเกินขนาดกรีดแขนพร้อม,Level 5,#โรคซึมเศร้า


In [ ]:
df['Label_Enc'] = df['Label (Specialist)'].str.replace('Level 1','Low Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 2','Mid Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 3','Mid Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 4','High Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 5','High Level Depress')

# Data Prep

In [ ]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|[^\s()]*?\)|)+(?:[^\s()]*?\)||[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def process_text(text):
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)

    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    return res

In [ ]:
df['Tweet'] = df['Tweet'].map(lambda x: ' '.join(process_text(x)))

In [ ]:
df.head(5)

,Unnamed: 0,DateTime,Uid,Tweet,Label (Specialist),HashTag,Label_Enc
0,0,18/5/2021 07:24,a38755f80aee,ถ้า ฉัน ตาย ไป จริงๆ ก็ คง ดี สิ นะ,Level 2,#โรคซึมเศร้า,Mid Level Depress
1,1,20/5/2021 16:06,a38755f80aee,เหนื่อย เหนื่อย จน จะ ตาย อยู่แล้ว มัน ไม กัน ...,Level 1,#โรคซึมเศร้า,Low Level Depress
2,2,16/6/2021 07:00,9b7d113b270a,พยายาม ที่จะ มีชีวิต อยู่ แต่ ก็ เหนื่อย กับ โ...,Level 1,#โรคซึมเศร้า,Low Level Depress
3,3,28/5/2021 18:43,0b981040c16d,ชีวิต ก็ มี อยู่ แค่นี้ อยาก หาย ไป แล้ว นะ,Level 1,#โรคซึมเศร้า,Low Level Depress
4,4,13/7/2021 18:06,937c5794646d,วัน วุ่น ๆ ของ วัยรุ่น กิน ยา เกินขนาด กรีด แข...,Level 5,#โรคซึมเศร้า,High Level Depress


In [ ]:
ws_tfidf = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), sublinear_tf=True)
ws_vec = ws_tfidf.fit_transform(df['Tweet']).toarray()
X = ws_vec

In [ ]:
conditions = [
    (df['Label_Enc'] == 'Other'),
    (df['Label_Enc'] == 'Low Level Depress'),
    (df['Label_Enc'] == 'Mid Level Depress'),
    (df['Label_Enc'] == 'High Level Depress'),
    ]

values = ['0', '1', '2', '3']

df['Label_Enc'] = np.select(conditions, values)

In [ ]:
y = df['Label_Enc']
''' removed due to wrong transformation
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
'''

' removed due to wrong transformation\nle = LabelEncoder()\nle.fit(y)\ny = le.transform(y)\n'

# Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Logistic Reg

In [ ]:
model = LogisticRegression(max_iter=10000,penalty='l2',C=1)
model.fit(X_train,y_train)

LogisticRegression(C=1, max_iter=10000)

In [ ]:
y_pred = model.predict(X_test)
y_true = np.array(y_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.36      0.49        61
           1       0.74      0.94      0.83       271
           2       0.67      0.39      0.50        76
           3       0.79      0.73      0.75       106

    accuracy                           0.75       514
   macro avg       0.75      0.61      0.64       514
weighted avg       0.75      0.75      0.73       514



# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf_not_tuning))

              precision    recall  f1-score   support

           0       0.74      0.38      0.50        61
           1       0.74      0.93      0.83       271
           2       0.52      0.34      0.41        76
           3       0.70      0.62      0.66       106

    accuracy                           0.71       514
   macro avg       0.68      0.57      0.60       514
weighted avg       0.70      0.71      0.69       514



# SGD Classifier
some user think that SGD is a classifier. But no, that's a linear classifier optimized by the SGD.
---



In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
SGDclf = SGDClassifier(loss='hinge',penalty='l2') # Linear SVM from TUNING PARAM
SGDclf.fit(X_train,y_train)

SGDclf_pred = SGDclf.predict(X_test)
print(classification_report(y_true, SGDclf_pred))

              precision    recall  f1-score   support

           0       0.70      0.46      0.55        61
           1       0.78      0.91      0.84       271
           2       0.54      0.42      0.47        76
           3       0.81      0.75      0.78       106

    accuracy                           0.75       514
   macro avg       0.71      0.64      0.66       514
weighted avg       0.74      0.75      0.74       514



# NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import MultinomialNB
Naive_clf = MultinomialNB()
Naive_clf.fit(X_train, y_train)
Naive_pred = Naive_clf.predict(X_test)
print(classification_report(y_true, Naive_pred))

              precision    recall  f1-score   support

           0       1.00      0.02      0.03        61
           1       0.62      1.00      0.76       271
           2       0.80      0.11      0.19        76
           3       0.89      0.54      0.67       106

    accuracy                           0.65       514
   macro avg       0.83      0.41      0.41       514
weighted avg       0.74      0.65      0.57       514



# Creating Custom Prediction Function

In [ ]:
def custom_pred(input_string: str) -> pd.DataFrame:
    rows = input_string.strip().split('\n')
    if len(rows) <= 20:
      print("รบกวนช่วยพิมพ์มากกว่านี้หน่อยได้ไหม...")
    elif len(rows) >= 50:
      df = pd.DataFrame(rows)
      df.iloc[:,0] = df.iloc[:,0].map(lambda x: ' '.join(process_text(x)))
      custom_text_input = ws_tfidf.transform(df.iloc[:,0]).toarray()

      predict_values = model.predict(custom_text_input)
    
      return predict_values
    # reverse encoder
    

In [ ]:
custom_pred('ควยโก้')

รบกวนช่วยพิมพ์มากกว่านี้หน่อยได้ไหม...
